## Relational Databases and Structured Query Language (SQL) Samantha Cress

Relational database management systems (RDBMSs) store data in tables and define relationships among the tables. Structured Query Language (SQL) is used almost universally with relational database systems to manipulate data and perform queries, which request information that satisfies given criteria.2

In [1]:
import pandas as pd

In [2]:
import sqlite3

Connecting to the Database in Python
To work with the database in Python, first call sqlite3’s connect function to connect to the database and obtain a Connection object:

In [6]:
connection = sqlite3.connect('books.db')

Viewing the authors Table’s Contents
Let’s use a SQL query and pandas to view the authors table’s contents:

In [7]:
pd.options.display.max_columns = 10

A SQL SELECT query gets rows and columns from one or more tables in a database. In the query:

In [8]:
pd.read_sql('SELECT * FROM authors', connection,
   ...:              index_col=['id'])

,first,last
id,,
1,Paul,Deitel
2,Harvey,Deitel
3,Abbey,Deitel
4,Dan,Quirk
5,Alexander,Wald


In [11]:
pd.read_sql('SELECT * FROM authors', connection,
   ...:              index_col=['id'])

,first,last
id,,
1,Paul,Deitel
2,Harvey,Deitel
3,Abbey,Deitel
4,Dan,Quirk
5,Alexander,Wald


In [12]:
pd.read_sql('SELECT * FROM titles', connection)

,isbn,title,edition,copyright
0,0135404673,Intro to Python for CS and DS,1,2020
1,0132151006,Internet & WWW How to Program,5,2012
2,0134743350,Java How to Program,11,2018
3,0133976890,C How to Program,8,2016
4,0133406954,Visual Basic 2012 How to Program,6,2014
5,0134601548,Visual C# How to Program,6,2017
6,0136151574,Visual C++ How to Program,2,2008
7,0134448235,C++ How to Program,10,2017
8,0134444302,Android How to Program,3,2017
9,0134289366,Android 6 for Programmers,3,2016


author_ISBN Table
The author_ISBN table uses the following columns to associate authors from the authors table with their books in the titles table:

id—An author’s id (an integer).
isbn—The book’s ISBN (a string).

The id column is a foreign key, which is a column in this table that matches a primary-key column in another table—in particular, the authors table’s id column.

In [13]:
df = pd.read_sql('SELECT * FROM author_ISBN', connection)

In [14]:
df.head()

,id,isbn
0,1,0134289366
1,2,0134289366
2,5,0134289366
3,1,0135404673
4,2,0135404673


Every foreign-key value must appear as the primary-key value in a row of another table so the DBMS can ensure that the foreign-key value is valid. This is known as the Rule of Referential Integrity. 

## 17.2.2 SELECT Queries Samantha Cress

To retrieve only specific columns, specify a comma-separated list of column names. For example, let’s retrieve only the columns first and last from the authors table:

In [15]:
pd.read_sql('SELECT first, last FROM authors', connection)

,first,last
0,Paul,Deitel
1,Harvey,Deitel
2,Abbey,Deitel
3,Dan,Quirk
4,Alexander,Wald


## 17.2.3 WHERE Clause Samantha Cress

SQL’s WHERE clause specifies a query’s selection criteria.

In [16]:
pd.read_sql("""SELECT title, edition, copyright
    ...:                FROM titles
    ...:                WHERE copyright > '2016'""", connection)

,title,edition,copyright
0,Intro to Python for CS and DS,1,2020
1,Java How to Program,11,2018
2,Visual C# How to Program,6,2017
3,C++ How to Program,10,2017
4,Android How to Program,3,2017


Pattern Matching: Zero or More Characters 
The WHERE clause may can contain the operators <, >, <=, >=, =, <> (not equal) and LIKE. Operator LIKE is used for pattern matching—searching for strings that match a given pattern. A pattern that contains the percent (%) wildcard character searches for strings that have zero or more characters at the percent character’s position in the pattern. 

In [17]:
pd.read_sql("""SELECT id, first, last
    ...:                FROM authors
    ...:                WHERE last LIKE 'D%'""",
    ...:             connection, index_col=['id'])

,first,last
id,,
1,Paul,Deitel
2,Harvey,Deitel
3,Abbey,Deitel


Pattern Matching: Any Character
An underscore (_) in the pattern string indicates a single wildcard character at that position. Let’s select the rows of all the authors whose last names start with any character, followed by the letter b, followed by any number of additional characters (specified by %):

In [18]:
pd.read_sql("""SELECT id, first, last
    ...:                FROM authors
    ...:                WHERE first LIKE '_b%'""",
    ...:             connection, index_col=['id'])

,first,last
id,,
3,Abbey,Deitel


## 17.2.4 ORDER BY Clause Samantha Cress

The ORDER BY clause sorts a query’s results into ascending order (lowest to highest) or descending order (highest to lowest), specified with ASC and DESC, respectively. The default sorting order is ascending, so ASC is optional. Let’s sort the titles in ascending order:

In [19]:
pd.read_sql('SELECT title FROM titles ORDER BY title ASC',
    ...:              connection)

,title
0,Android 6 for Programmers
1,Android How to Program
2,C How to Program
3,C++ How to Program
4,Internet & WWW How to Program
5,Intro to Python for CS and DS
6,Java How to Program
7,Visual Basic 2012 How to Program
8,Visual C# How to Program
9,Visual C++ How to Program


Sorting By Multiple Columns
To sort by multiple columns, specify a comma-separated list of column names after the ORDER BY keywords. Let’s sort the authors’ names by last name, then by first name for any authors who have the same last name:

In [20]:
pd.read_sql("""SELECT id, first, last
    ...:                FROM authors
    ...:                ORDER BY last, first""",
    ...:               connection, index_col=['id'])

,first,last
id,,
3,Abbey,Deitel
2,Harvey,Deitel
1,Paul,Deitel
4,Dan,Quirk
5,Alexander,Wald


The sorting order can vary by column. Let’s sort the authors in descending order by last name and ascending order by first name for any authors who have the same last name:

In [21]:
pd.read_sql("""SELECT id, first, last
    ...:                FROM authors
    ...:                ORDER BY last DESC, first ASC""",
    ...:              connection, index_col=['id'])

,first,last
id,,
5,Alexander,Wald
4,Dan,Quirk
3,Abbey,Deitel
2,Harvey,Deitel
1,Paul,Deitel


Combining the WHERE and ORDER BY Clauses
The WHERE and ORDER BY clauses can be combined in one query. Let’s get the isbn, title, edition and copyright of each book in the titles table that has a title ending with 'How to Program' and sort them in ascending order by title.

In [22]:
pd.read_sql("""SELECT isbn, title, edition, copyright
    ...:                FROM titles
    ...:                WHERE title LIKE '%How to Program'
    ...:                ORDER BY title""", connection)

,isbn,title,edition,copyright
0,0134444302,Android How to Program,3,2017
1,0133976890,C How to Program,8,2016
2,0134448235,C++ How to Program,10,2017
3,0132151006,Internet & WWW How to Program,5,2012
4,0134743350,Java How to Program,11,2018
5,0133406954,Visual Basic 2012 How to Program,6,2014
6,0134601548,Visual C# How to Program,6,2017
7,0136151574,Visual C++ How to Program,2,2008


## 17.2.5 Merging Data from Multiple Tables: INNER JOIN

You can merge data from multiple tables, referred to as joining the tables, with INNER JOIN. Let’s produce a list of authors accompanied by the ISBNs for books written by each author—because there are many results for this query, we show just the head of the result:

In [23]:
pd.read_sql("""SELECT first, last, isbn
    ...:                FROM authors
    ...:                INNER JOIN author_ISBN
    ...:                    ON authors.id = author_ISBN.id
    ...:                ORDER BY last, first""", connection).head()

,first,last,isbn
0,Abbey,Deitel,0132151006
1,Abbey,Deitel,0133406954
2,Harvey,Deitel,0134289366
3,Harvey,Deitel,0135404673
4,Harvey,Deitel,0132151006


The INNER JOIN’s ON clause uses a primary-key column in one table and a foreign-key column in the other to determine which rows to merge from each table. This query merges the authors table’s first and last columns with the author_ISBN table’s isbn column and sorts the results in ascending order by last then first.

## 17.2.6 INSERT INTO Statement Samantha Cress

The INSERT INTO statement inserts a row into a table. Let’s insert a new author named Sue Red into the authors table by calling Cursor method execute, which executes its SQL argument and returns the Cursor:

In [25]:
cursor = connection.cursor()

In [26]:
cursor = cursor.execute("""INSERT INTO authors (first, last)
    ...:                            VALUES ('Sue', 'Red')""")

In [27]:
pd.read_sql('SELECT id, first, last FROM authors',
    ...:             connection, index_col=['id'])

,first,last
id,,
1,Paul,Deitel
2,Harvey,Deitel
3,Abbey,Deitel
4,Dan,Quirk
5,Alexander,Wald
6,Sue,Red


Note Regarding Strings That Contain Single Quotes
SQL delimits strings with single quotes ('). A string containing a single quote, such as O’Malley, must have two single quotes in the position where the single quote appears (e.g., 'O''Malley'). The first acts as an escape character for the second. Not escaping single-quote characters in a string that’s part of a SQL statement is a SQL syntax error.

## 17.2.7 UPDATE Statement Samantha Cress

An UPDATE statement modifies existing values. Let’s assume that Sue Red’s last name is incorrect in the database and update it to 'Black':

In [28]:
cursor = cursor.execute("""UPDATE authors SET last='Black'
    ...:                            WHERE last='Red' AND first='Sue'""")

In [29]:
cursor.rowcount

1

In [30]:
pd.read_sql('SELECT id, first, last FROM authors',
    ...:             connection, index_col=['id'])

,first,last
id,,
1,Paul,Deitel
2,Harvey,Deitel
3,Abbey,Deitel
4,Dan,Quirk
5,Alexander,Wald
6,Sue,Black


## 17.2.8 DELETE FROM Statement Samantha Cress

In [31]:
cursor = cursor.execute('DELETE FROM authors WHERE id=6')

In [32]:
cursor.rowcount

1

In [33]:
pd.read_sql('SELECT id, first, last FROM authors',
    ...:             connection, index_col=['id'])

,first,last
id,,
1,Paul,Deitel
2,Harvey,Deitel
3,Abbey,Deitel
4,Dan,Quirk
5,Alexander,Wald


Closing the Database
When you no longer need access to the database, you should call the Connection’s close method to disconnect from the database
connection.close()

Self Check: Select from the titles table all the titles and their edition numbers in descending order by edition number. Show only the first three results.

In [34]:
pd.read_sql("""SELECT title, edition FROM titles
   ...:                 ORDER BY edition DESC""", connection).head(3)

,title,edition
0,Java How to Program,11
1,C++ How to Program,10
2,C How to Program,8


Self Check: Select from the authors table all authors whose first names start with 'A'.

In [35]:
pd.read_sql("""SELECT * FROM authors
    ...:                WHERE first LIKE 'A%'""", connection)

,id,first,last
0,3,Abbey,Deitel
1,5,Alexander,Wald


Self Check: SQL’s NOT keyword reverses the value of a WHERE clause’s condition. Select from the titles table all titles that do not end with 'How to Program'.

In [36]:
pd.read_sql("""SELECT isbn, title, edition, copyright
    ...:                FROM titles
    ...:                WHERE title NOT LIKE '%How to Program'
    ...:                ORDER BY title""", connection)

,isbn,title,edition,copyright
0,0134289366,Android 6 for Programmers,3,2016
1,0135404673,Intro to Python for CS and DS,1,2020
